In [10]:
!pip install --pre unified-planning==1.0.0.95.dev1
!pip install up_fast_downward==0.3.1

In [11]:
!pip install matplotlib==3.7.1  # for visualisation in this notebook

In [13]:
from unified_planning.shortcuts import *

import unified_planning as up

up.shortcuts.get_environment().credits_stream = None

## Part (a): Model the Task

The code below models a simpler version of the Household Robot domain with two rooms and one open door between them. The robot is in room A initially and needs to move to room B. You can use it as a starting point for your own solution to the full Household Robot task.

In [14]:

def get_planning_task():
    # Declare user types.
    Room = UserType("Room")
    Door = UserType("Door")
    Key = UserType("Key")

    # Declare predicates.
    robot_in = up.model.Fluent("robot_in", BoolType(), r=Room)
    connected = up.model.Fluent("connected", BoolType(), r1=Room, d=Door, r2=Room)
    room_has_door = up.model.Fluent("room_has_door", BoolType(), r=Room, d=Door)
    door_is_open = up.model.Fluent("door_is_open", BoolType(), d=Door)
    room_has_key = up.model.Fluent("room_has_key", BoolType(), r=Room, k=Key) #which keys are present in room
    robot_has_key = up.model.Fluent("robot_has_key", BoolType(), k=Key) #which key the robot has picked
    door_key_match = up.model.Fluent("door_key_match", BoolType(), d=Door, k=Key) #which key open which door

    # Add (typed) objects to problem.
    problem = up.model.Problem("household")

    def get_room(room):
        return up.model.Object(f"room{room}", Room)

    def get_door(door):
        return up.model.Object(f"door{door}", Door)

    def get_key(key):
        return up.model.Object(f"key{key}", Key)

    roomLivingRoom = get_room("LivingRoom")
    roomCorridor = get_room("Corridor")
    roomKitchen = get_room("Kitchen")
    roomBathroom = get_room("Bathroom")
    roomLobby = get_room("Lobby")
    rooms = [roomLivingRoom, roomCorridor, roomKitchen, roomBathroom, roomLobby]

    doorLivingRoom = get_door("LivingRoom")
    doorKitchen = get_door("Kitchen")
    doorCorridor = get_door("Corridor")
    doorBathroom = get_door("Bathroom")
    doorLobby = get_door("Lobby")
    doors = [doorLivingRoom, doorKitchen, doorCorridor, doorBathroom, doorLobby]


    keyLivingRoom = get_key("LivingRoom")
    keyKitchen = get_key("Kitchen")
    keyCorridor = get_key("Corridor")
    keyBathroom = get_key("Bathroom")
    keyLobby = get_key("Lobby")
    keys = [keyLivingRoom, keyKitchen, keyCorridor, keyBathroom, keyLobby]


    problem.add_objects(rooms)
    problem.add_objects(doors)
    problem.add_objects(keys)

    connections = [
        (roomLivingRoom, doorKitchen, roomKitchen),
        (roomLivingRoom, doorLivingRoom, roomCorridor),
        (roomCorridor, doorBathroom, roomBathroom),
        (roomCorridor, doorCorridor, roomLobby)
    ]

    rooms_and_doors = []
    for room1, door, room2 in connections:
        rooms_and_doors.append((room1, door))
        rooms_and_doors.append((room2, door))

    rooms_and_doors.append((roomLobby, doorLobby))

    keys_locations = [
        (roomLivingRoom, keyLivingRoom),
        (roomLivingRoom, keyKitchen),
        (roomCorridor, keyBathroom),
        (roomBathroom, keyLobby),
        (roomBathroom, keyCorridor)
    ]

    door_keys = [
        (doorLivingRoom, keyLivingRoom),
        (doorKitchen, keyKitchen),
        (doorCorridor, keyCorridor),
        (doorBathroom, keyBathroom),
        (doorLobby, keyLobby)
    ]


    # Specify the initial state.
    problem.add_fluent(robot_in, default_initial_value=False)
    problem.add_fluent(connected, default_initial_value=False)
    problem.add_fluent(room_has_door, default_initial_value=False)
    problem.add_fluent(door_is_open, default_initial_value=False)
    problem.add_fluent(robot_has_key, default_initial_value=False)
    problem.add_fluent(door_key_match, default_initial_value=False)
    problem.add_fluent(room_has_key, default_initial_value=False)
    problem.set_initial_value(robot_in(roomLivingRoom), True)

    for room1, door, room2 in connections:
        problem.set_initial_value(connected(room1, door, room2), True)
        problem.set_initial_value(connected(room2, door, room1), True)

    for room, door in rooms_and_doors:
        problem.set_initial_value(room_has_door(room, door), True)

    for room, key in keys_locations:
        problem.set_initial_value(room_has_key(room, key), True)

    for door, key in door_keys:
        problem.set_initial_value(door_key_match(door, key), True)

    # Add actions.
    move = up.model.InstantaneousAction("move", room1=Room, door=Door, room2=Room)
    room1 = move.parameter("room1")
    door = move.parameter("door")
    room2 = move.parameter("room2")
    move.add_precondition(robot_in(room1))
    move.add_precondition(connected(room1, door, room2))
    move.add_precondition(door_is_open(door))
    move.add_effect(robot_in(room1), False)
    move.add_effect(robot_in(room2), True)
    problem.add_action(move)

    open = up.model.InstantaneousAction("open", room=Room, door=Door, key=Key)
    room = open.parameter("room")
    door = open.parameter("door")
    key = open.parameter("key")
    open.add_precondition(robot_in(room))
    open.add_precondition(robot_has_key(key))
    open.add_precondition(room_has_door(room, door))
    open.add_precondition(door_key_match(door, key))
    open.add_effect(door_is_open(door), True)
    problem.add_action(open)

    pick = up.model.InstantaneousAction("pick", room=Room, key=Key)
    room = pick.parameter("room")
    key = pick.parameter("key")
    pick.add_precondition(robot_in(room))
    pick.add_precondition(room_has_key(room, key))
    pick.add_effect(room_has_key(room, key), False)
    pick.add_effect(robot_has_key(key), True)
    problem.add_action(pick)

    # Specify the goal.
    problem.add_goal(door_is_open(doorLobby))

    # We want to minimize the plan cost.
    problem.add_quality_metric(MinimizeActionCosts({}, default=Int(1)))
    return problem

problem = get_planning_task()


## Part (b): Find a (possibly suboptimal) plan

Solve the task with greedy best-first search using the FF heuristic. The example code below uses the h^add heuristic. You need to inspect the output to stdout to see the heuristic value of the initial state.

In [7]:
params = {
    "fast_downward_search_config": "eager_greedy([ff()])",
    "log_level": "info"
}

with OneshotPlanner(name="fast-downward", params=params) as planner:
    result = planner.solve(problem, output_stream=sys.stdout)
    if result.status == up.engines.PlanGenerationResultStatus.SOLVED_SATISFICING:
        print("Found a plan of length:", len(result.plan.actions))
        print(result.plan)
        with PlanValidator() as validator:
            val_result = validator.validate(problem, result.plan)
            print("Plan cost:", val_result.metric_evaluations)
    else:
        print("No plan found.", result.log_messages)

INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.
INFO     translator stdin: None
INFO     translator time limit: None
INFO     translator memory limit: None
INFO     translator command line string: /usr/bin/python3 /usr/local/lib/python3.10/dist-packages/up_fast_downward/downward/builds/release/bin/translate/translate.py /tmp/tmpwubk7025/domain.pddl /tmp/tmpwubk7025/problem.pddl --sas-file output.sas
Parsing...
Parsing: [0.000s CPU, 0.002s wall-clock]
Normalizing task... [0.000s CPU, 0.000s wall-clock]
Instantiating...
Generating Datalog program... [0.000s CPU, 0.001s wall-clock]
Normalizing Datalog program...
Normalizing Datalog program: [0.000s CPU, 0.002s wall-clock]
Preparing model... [0.000s CPU, 0.001s wall-clock]
Generated 18 rules.
Computing model... [0.010s CPU, 0.001s wall-clock]
110 relevant atoms
69 auxiliary atoms
179 final queue length
187 total queue pushes
Completing instantiation... [0.000s CPU, 0.001s wall-clock]
Inst

# Planning Lab

## Part (c): Find an optimal plan

Solve the task with A* using the `iPDB` heuristic.

In [15]:
params = {
    "fast_downward_search_config": "astar(ipdb())",
    "log_level": "info"
}

with OneshotPlanner(name="fast-downward", params=params) as planner:
    result = planner.solve(problem, output_stream=sys.stdout)
    if result.status == up.engines.PlanGenerationResultStatus.SOLVED_SATISFICING:
        print("Found a plan of length:", len(result.plan.actions))
        print(result.plan)
        with PlanValidator() as validator:
            val_result = validator.validate(problem, result.plan)
            print("Plan cost:", val_result.metric_evaluations)
    else:
        print("No plan found.")

INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.
INFO     translator stdin: None
INFO     translator time limit: None
INFO     translator memory limit: None
INFO     translator command line string: /usr/bin/python3 /usr/local/lib/python3.10/dist-packages/up_fast_downward/downward/builds/release/bin/translate/translate.py /tmp/tmpw1swuhj1/domain.pddl /tmp/tmpw1swuhj1/problem.pddl --sas-file output.sas
Parsing...
Parsing: [0.000s CPU, 0.003s wall-clock]
Normalizing task... [0.000s CPU, 0.000s wall-clock]
Instantiating...
Generating Datalog program... [0.000s CPU, 0.001s wall-clock]
Normalizing Datalog program...
Normalizing Datalog program: [0.000s CPU, 0.001s wall-clock]
Preparing model... [0.000s CPU, 0.001s wall-clock]
Generated 18 rules.
Computing model... [0.010s CPU, 0.001s wall-clock]
110 relevant atoms
69 auxiliary atoms
179 final queue length
187 total queue pushes
Completing instantiation... [0.000s CPU, 0.001s wall-clock]
Inst